# California PPE Projections

## California Burn Rates

Burn rates taken from the CHA memo.

In [7]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells

config = set_config('../config/ca-vent')}
restart = RestartModel(config_dir='../config/ca-vent', population='dict')
model = restart.model

burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
display(burn_sheet)

SyntaxError: unmatched '}' (<ipython-input-7-d28f5451c487>, line 8)

In [8]:
import ipywidgets as widgets
from scipy.stats import norm
import math

def triangular(a,b,c):
    return math.sqrt( ((a*a + b*b + c*c) - a*b - a*c - b*c) / 18 )


ca_epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
stdev = ca_epi_df.apply(
    lambda row: triangular(
        row['Vent Mid'],
        row['Vent Low'], 
        row['Vent High']
    ), axis=1
)

ca_epi_df['Vent Mean'] = ca_epi_df.mean(axis=1)

ca_epi_df['Vent SD'] = stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Vent EOQ'] = df['Vent Mean'] + (Z * df['Vent SD'])
    return df

def display_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(ca_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_df.loc['CA Projected Non-COVID Patients', 'Vent EOQ'] = 0
    eoq_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (model.demand.demand_by_pop_per_person_pn_uc.array.T * ca_epi_df["Vent EOQ"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.drop(['Ventilators', 'Surgical Masks'], axis=1, inplace=True)
    preds_df.df.drop(['CA Projected Non-COVID Surge Patients'], axis=0, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    display_population(preds_sheet, round=True)

cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
out = widgets.interactive_output(display_eoq, {'cr': cr_slider, 'days': day_slider})
widgets.VBox([cr_slider, day_slider, out])

## OES Projections

Using WA tiering system, BLS OES population statistics, and WA burn rate assumptions.

In [9]:
oes_config = set_config('../restart')
oes_restart = RestartModel(
    config_dir='../restart',
    data_dir='../../data/ingestion',
    population='oes',
    state='California',
    subpop='healthcare')
oes_model = oes_restart.model
oes_model.inventory.set_average_orders_per_period(oes_model.demand.demand_by_popsum1_total_rp1n_tc)

oes_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def set_stock(backstop):
    backstop = [backstop]
    oes_model.inventory.order(oes_model.inventory.inv_by_popsum1_total_rp1n_tc)
    oes_model.inventory.set_min_in_periods(backstop)
    display_stock(oes_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(oes_model.demand.demand_per_unit_map_dn_um.df))
oes_pop = format_population(to_sheet(oes_model.population.population_pP_tr.df))
    
oes_out = widgets.interactive_output(dashboard, {'backstop': oes_slider})

widgets.VBox([oes_slider, oes_out])

## JHU Burn Rates

### ICU Assumptions

Gloves: 2 gloves for each of 170 changes per patient per day. This assumes a change with each patient encounter, as per normal practice by all healthcare workers.

Gowns: Assumes that COVID patients are cohorted and that a single gown is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. Accounting for all the various healthcare workers involved in the care of an ICU patient, 20 changes per patient per day.

Simple masks: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 10 changes per patient per day.

N95 respirators: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. N95s are used only by healthcare workers in proximity (3 feet) to COVID patients. 6 changes per patient per day.

### Non-ICU Assumptions

Gloves: 2 gloves for each of 80 changes per patient per day. This assumes a change with each patient encounter, as per normal practice by all healthcare workers.

Gowns: Assumes that COVID patients are cohorted and that a single gown is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 20 changes per patient per day.

Simple masks: Assumes that COVID patients are cohorted and that a single mask is worn for 4 hours by each healthcare worker assigned to the COVID cohort, unless it becomes visibly soiled. 10 changes per patient per day.

N95 respirators: Respirators worn only for nebulizer treatments, intubations, and other aerosol-generating procedures. An average of 2.6 changes per patient per day.

### Daily Burn Rates

Let `n` be the number of hospital COVID patients. IHME assumption is that 27% of patients will go to ICU.

Gloves (g): 

`g = (0.27n)(85) + (0.73n)(40) = 52.15 pairs per patient per day`

Gowns: (G): 

`G = 20n = 20 gowns per patient per day`

Simple masks (m): 

`m = 10n = 10 masks per patient per day`

N95 Respirators (M): 

`M = (0.27n)(6) + (0.73n)(2.6) = 3.52 N95 masks per patient per day`





In [12]:
jhu_config = set_config('../config/jhu')
jhu_restart = RestartModel(config_dir='../config/jhu', population='dict')
jhu_model = jhu_restart.model

epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
epi_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)

# adjusting so it's in terms of population, since we're not doing vents here
epi_df = epi_df * 4.12
epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
stdev = epi_df.apply(
    lambda row: triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

epi_df['Population Mean'] = epi_df.mean(axis=1)

epi_df['Population SD'] = stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", jhu_config)
    preds_df.array = (jhu_model.demand.demand_by_pop_per_person_pn_uc.array.T * epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_sheet = to_sheet(preds_df.df)
    display_population(preds_sheet, round=True)
    
jhu_burn_sheet  = format_population(to_sheet(jhu_model.demand.demand_per_unit_map_dn_um.df))

jhu_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
jhu_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
jhu_out = widgets.interactive_output(display_eoq, {'cr': jhu_cr_slider, 'days': jhu_day_slider})
widgets.VBox([jhu_cr_slider, jhu_day_slider, jhu_out])

In [13]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Vent Analysis']),
    v.Tab(children=['Population']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[
                    "Vent + PPE analysis with CA burn rates:"
                ]),
                cr_slider, day_slider, out,
                v.CardText(children=[
                    "PPE and population analysis with JHU burn rates:"
                ]),
                jhu_cr_slider, jhu_day_slider, jhu_out,
                v.CardText(children=[
                    "PPE and population analysis with OES population and WA burn rates:"
                ]),
                oes_slider, oes_out,
            ]),
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["BLS Population with High Granularity"]),
                v.CardText(children=[
                    "OES Population"
                ]),
                oes_pop
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand: CA Assumptions"]),
                burn_sheet,
                v.CardSubtitle(children=["Per Capita Resource Demand: JHU Assumptions"]),
                jhu_burn_sheet,
                v.CardSubtitle(children=["Per Capita Resource Demand: WA Assumptions"]),
                wa_burn_sheet,
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['Vent Analysis']), Tab(children=['Population']), Tab(children=['Burn Rates']), Ta…